In [12]:
#!/usr/bin/env python3
"""
multi_indicator_scanner.py

Self-contained scanner (no CLI). Configure at the top, run directly.
Saves outputs to outputs/YYYY-MM-DD/
Fixed ADX index/length mismatch bug.
Added SMA(30) with toggleable signal.
Added Telegram notifications for trades to take (BUY signals).
"""

import os
import datetime as dt
from typing import Optional, Dict, Any, List
import pandas as pd
import numpy as np
import yfinance as yf

# Optional third-party: requests (for Telegram). Falls back to print if unavailable.
try:
    import requests  # pip install requests
except Exception:
    requests = None

# Optional: nice timezone stamp for IST; safe fallback if not present.
try:
    from zoneinfo import ZoneInfo  # Python 3.9+
except Exception:
    ZoneInfo = None

# Try to import user's custom indicators (optional)
try:
    from custom_indicators import calculate_ema as cust_calc_ema, calculate_supertrend as cust_calc_supertrend
    HAVE_CUSTOM_IND = True
except Exception:
    HAVE_CUSTOM_IND = False
    cust_calc_ema = None
    cust_calc_supertrend = None


# -------------------------------
# ----- User Configuration ------
# -------------------------------
STOCKS_TO_SCAN = [
    'SBICARD.NS', 'BDL.NS', 'INDHOTEL.NS', 'BSE.NS', 'NYKAA.NS', 'BAJFINANCE.NS',
    'PAYTM.NS', 'SOLARINDS.NS', 'CHOLAFIN.NS', 'UNITDSPR.NS', 'DIVISLAB.NS',
    'MUTHOOTFIN.NS', 'BHARTIARTL.NS', 'ICICIBANK.NS', 'MAZDOCK.NS', 'SHREECEM.NS',
    'DIXON.NS', 'PERSISTENT.NS', 'SRF.NS', 'TVSMOTOR.NS', 'SBILIFE.NS', 'MAXHEALTH.NS',
    'MFSL.NS', 'COFORGE.NS', 'HDFCLIFE.NS', 'INDIGO.NS', 'KOTAKBANK.NS',
    'HDFCBANK.NS', 'BEL.NS', 'BAJAJFINSV.NS'
]

# Choose evaluation date (YYYY-MM-DD) or None for most recent
EVAL_DATE = None  # e.g. "2025-09-01" or None for latest

# Where to save results
OUTPUT_ROOT = "outputs"

# Indicator and strategy settings
CONFIG: Dict[str, Any] = {
    "INDICATORS": {
        "EMA_CROSS": True,
        "SUPERTREND": True,
        "RSI": False,
        "ADX": False,
        "MACD": False,
        "BOLLINGER": False,
        "SMA": False,       # 20/50 crossover
        "SMA30": True,     # Close crossing above SMA(30)
        "VOL_SMA": False,
        "OBV": False,
    },
    "COMBINE_MODE": "all",  # 'all', 'any', or 'majority'

    # Indicator params
    "ST_LENGTH": 10,
    "ST_MULTIPLIER": 3.0,
    "EMA_FAST_LENGTH": 9,
    "EMA_SLOW_LENGTH": 15,
    "SMA_FAST_LENGTH": 20,
    "SMA_SLOW_LENGTH": 50,
    "SMA30_LENGTH": 30,     # NEW
    "RSI_LENGTH": 14,
    "RSI_BUY_THRESHOLD": 40,
    "ADX_LENGTH": 14,
    "ADX_THRESHOLD": 25,
    "MACD_FAST": 12,
    "MACD_SLOW": 26,
    "MACD_SIGNAL": 9,
    "BOLLINGER_LENGTH": 20,
    "BOLLINGER_STD": 2,
    "VOL_SMA_LENGTH": 20,
    "VOL_MULTIPLIER": 1.5,

    # Data settings
    "MIN_ROWS": 30,
    "YF_PERIOD_DAYS": 200,

    # Telegram settings
    "TELEGRAM": {
        "ENABLE": True,              # If True, try to send to Telegram; falls back to print when creds missing
        "BOT_TOKEN": "8230320738:AAEmc-U4jmsTFX6C_FszU6eLO2yu4qgaxJs",             # Leave empty to read from env TELEGRAM_BOT_TOKEN
        "CHAT_ID": "-4815813126",               # Leave empty to read from env TELEGRAM_CHAT_ID
        "PARSE_MODE": "HTML",        # 'HTML' is robust; avoids Markdown escaping headaches
        "SEND_SUMMARY": True,        # Send one summary message listing all BUY signals
        "SEND_EACH_TRADE": False,    # Additionally send one message per trade
        "DISABLE_WEB_PREVIEW": True, # Don't expand links (we don't send links anyway)
        "DRY_RUN": False,            # If True, never call Telegram API; just print what would be sent
        "CHUNK_CHARS": 3500          # Telegram max ~4096; we keep below limit
    }
}

VERBOSE = True


# -------------------------------
# ---- Indicator Functions ------
# -------------------------------
def ema(series, span): return series.ewm(span=span, adjust=False).mean()
def sma(series, length): return series.rolling(window=length, min_periods=1).mean()

def rsi(series, length=14):
    delta = series.diff()
    up, down = delta.clip(lower=0), -delta.clip(upper=0)
    ma_up = up.ewm(alpha=1/length, adjust=False).mean()
    ma_down = down.ewm(alpha=1/length, adjust=False).mean()
    rs = ma_up / (ma_down + 1e-9)
    return 100 - (100 / (1 + rs))

def compute_atr(df, length=14):
    tr = pd.concat([
        df['High'] - df['Low'],
        (df['High'] - df['Close'].shift()).abs(),
        (df['Low'] - df['Close'].shift()).abs()
    ], axis=1).max(axis=1)
    return tr.rolling(length, min_periods=1).mean()

def supertrend_fallback(df, period=10, multiplier=3.0):
    hl2 = (df['High'] + df['Low']) / 2
    atr = compute_atr(df, period)
    upperband, lowerband = hl2 + multiplier*atr, hl2 - multiplier*atr
    final_upper, final_lower = upperband.copy(), lowerband.copy()
    # ensure numeric indexable (avoid SettingWithCopy warnings)
    final_upper = final_upper.reset_index(drop=True)
    final_lower = final_lower.reset_index(drop=True)
    close_vals = df['Close'].reset_index(drop=True)

    for i in range(1, len(df)):
        final_upper.iat[i] = upperband.iat[i] if (upperband.iat[i] < final_upper.iat[i-1]) or (close_vals.iat[i-1] > final_upper.iat[i-1]) else final_upper.iat[i-1]
        final_lower.iat[i] = lowerband.iat[i] if (lowerband.iat[i] > final_lower.iat[i-1]) or (close_vals.iat[i-1] < final_lower.iat[i-1]) else final_lower.iat[i-1]

    supertrend = pd.Series(index=df.index, dtype=float)
    direction = pd.Series(index=df.index, dtype=int)
    for i in range(len(df)):
        if i == 0 or df['Close'].iat[i] <= final_upper.iat[i]:
            supertrend.iat[i] = final_upper.iat[i]
            direction.iat[i] = -1
        else:
            supertrend.iat[i] = final_lower.iat[i]
            direction.iat[i] = 1

    df = df.copy()
    df['supertrend'] = supertrend
    df['supertrend_direction'] = direction
    return df

def macd(series, fast=12, slow=26, signal=9):
    fast_ema, slow_ema = series.ewm(span=fast, adjust=False).mean(), series.ewm(span=slow, adjust=False).mean()
    macd_line = fast_ema - slow_ema
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    return pd.DataFrame({"macd": macd_line, "signal": signal_line, "hist": macd_line - signal_line}, index=series.index)

def adx(df, length=14):
    """
    Fixed ADX: returns pd.Series aligned with df.index.
    """
    high = df['High']
    low = df['Low']
    close = df['Close']

    up_move = high.diff()
    down_move = low.diff().mul(-1)

    plus_dm = pd.Series(np.where((up_move > down_move) & (up_move > 0), up_move, 0.0), index=df.index)
    minus_dm = pd.Series(np.where((down_move > up_move) & (down_move > 0), down_move, 0.0), index=df.index)

    tr = pd.concat([
        (high - low).abs(),
        (high - close.shift()).abs(),
        (low - close.shift()).abs()
    ], axis=1).max(axis=1)

    atr = tr.rolling(length, min_periods=1).mean()
    plus_dm_sum = plus_dm.rolling(length, min_periods=1).sum()
    minus_dm_sum = minus_dm.rolling(length, min_periods=1).sum()

    plus_di = 100 * (plus_dm_sum / (atr + 1e-9))
    minus_di = 100 * (minus_dm_sum / (atr + 1e-9))

    dx = (plus_di - minus_di).abs() / (plus_di + minus_di + 1e-9) * 100
    adx_series = dx.rolling(length, min_periods=1).mean()
    adx_series.name = 'ADX'
    return adx_series

def obv(df):
    sign = df['Close'].diff().fillna(0).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    return (sign * df['Volume']).cumsum()


# -------------------------------
# ---- Telegram Helpers ---------
# -------------------------------
def _now_ist_str():
    try:
        if ZoneInfo:
            return dt.datetime.now(ZoneInfo("Asia/Kolkata")).strftime("%Y-%m-%d %H:%M")
    except Exception:
        pass
    return dt.datetime.now().strftime("%Y-%m-%d %H:%M")

def _escape_html(s: str) -> str:
    return (str(s).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;"))

def resolve_telegram_creds(tcfg: Dict[str, Any]):
    token = tcfg.get("BOT_TOKEN") or os.getenv("TELEGRAM_BOT_TOKEN", "")
    chat_id = tcfg.get("CHAT_ID") or os.getenv("TELEGRAM_CHAT_ID", "")
    return token, chat_id

def _chunk_text(text: str, limit: int) -> List[str]:
    if len(text) <= limit:
        return [text]
    chunks, cur = [], []
    cur_len = 0
    for line in text.splitlines(True):
        if cur_len + len(line) > limit and cur:
            chunks.append("".join(cur))
            cur, cur_len = [line], len(line)
        else:
            cur.append(line); cur_len += len(line)
    if cur:
        chunks.append("".join(cur))
    return chunks

def send_telegram_message(text: str, tcfg: Dict[str, Any]):
    token, chat_id = resolve_telegram_creds(tcfg)
    parse_mode = tcfg.get("PARSE_MODE", "HTML")
    disable_preview = tcfg.get("DISABLE_WEB_PREVIEW", True)
    dry_run = tcfg.get("DRY_RUN", False)
    chunk_chars = int(tcfg.get("CHUNK_CHARS", 3500))

    if not tcfg.get("ENABLE", True):
        if VERBOSE: print("[Telegram] Disabled. Message:\n", text)
        return

    if dry_run or (not token) or (not chat_id) or (requests is None):
        # Safe fallback
        print("[Telegram] (dry-run/creds-missing/requests-missing) Would send:\n", text)
        return

    base_url = f"https://api.telegram.org/bot{token}/sendMessage"
    for piece in _chunk_text(text, chunk_chars):
        try:
            resp = requests.post(
                base_url,
                data={
                    "chat_id": chat_id,
                    "text": piece,
                    "parse_mode": parse_mode,
                    "disable_web_page_preview": "true" if disable_preview else "false",
                },
                timeout=15,
            )
            if resp.status_code != 200:
                print(f"[Telegram] Error {resp.status_code}: {resp.text[:200]}")
        except Exception as e:
            print(f"[Telegram] Exception sending message: {e}")

def build_summary_message(results_df: pd.DataFrame, details_df: pd.DataFrame, config: Dict[str, Any], run_date_str: str) -> str:
    ist_now = _now_ist_str()
    total = len(results_df)
    wins = int((results_df["decision"] == True).sum()) if not results_df.empty else 0

    header = f"<b>Scanner BUY signals — {run_date_str} (IST {ist_now})</b>\n"
    header += f"Scanned: <b>{total}</b> | Signals: <b>{wins}</b> | Mode: <b>{_escape_html(config.get('COMBINE_MODE','all'))}</b>\n\n"

    if wins == 0:
        return header + "No BUY signals today."

    # Create quick lookup for indicator booleans
    detail_map = {}
    if not details_df.empty and "ticker" in details_df.columns:
        for _, r in details_df.iterrows():
            detail_map[r["ticker"]] = r.to_dict()

    enabled_inds = [k for k, v in config["INDICATORS"].items() if v]

    lines = []
    buys = results_df[results_df["decision"] == True].copy()
    # Ensure consistent types for printing
    if "last_bar_date" in buys.columns:
        buys["last_bar_date"] = buys["last_bar_date"].astype(str)

    for _, rec in buys.iterrows():
        t = rec["ticker"]
        price = rec.get("last_close", None)
        when = rec.get("last_bar_date", "")
        drow = detail_map.get(t, {})

        # Build badges for enabled indicators (✓/✗)
        badges = []
        for ind in enabled_inds:
            val = drow.get(ind, None)
            check = "✅" if (val is True) else ("❌" if (val is False) else "·")
            # Friendlier label for SMA crossover
            label = "SMA20/50" if ind == "SMA" else ind
            badges.append(f"{_escape_html(label)}{check}")

        line = f"• <b>{_escape_html(t)}</b> @ <b>{price:.2f}</b> | {when}\n    <code>{'  '.join(badges)}</code>"
        lines.append(line)

    return header + "\n".join(lines)

def build_per_trade_messages(results_df: pd.DataFrame, details_df: pd.DataFrame, config: Dict[str, Any], run_date_str: str) -> List[str]:
    msgs = []
    if results_df.empty: return msgs
    enabled_inds = [k for k, v in config["INDICATORS"].items() if v]

    detail_map = {}
    if not details_df.empty and "ticker" in details_df.columns:
        for _, r in details_df.iterrows():
            detail_map[r["ticker"]] = r.to_dict()

    buys = results_df[results_df["decision"] == True].copy()
    if "last_bar_date" in buys.columns:
        buys["last_bar_date"] = buys["last_bar_date"].astype(str)

    for _, rec in buys.iterrows():
        t = rec["ticker"]
        price = rec.get("last_close", None)
        when = rec.get("last_bar_date", "")
        drow = detail_map.get(t, {})

        badges = []
        for ind in enabled_inds:
            val = drow.get(ind, None)
            check = "✅" if (val is True) else ("❌" if (val is False) else "·")
            label = "SMA20/50" if ind == "SMA" else ind
            badges.append(f"{_escape_html(label)}{check}")

        text = (
            f"<b>BUY Signal</b> — {_escape_html(run_date_str)}\n"
            f"Ticker: <b>{_escape_html(t)}</b>\n"
            f"Last Close: <b>{price:.2f}</b>\n"
            f"Bar: {when}\n"
            f"<code>{'  '.join(badges)}</code>"
        )
        msgs.append(text)
    return msgs


# -------------------------------
# ---- Core Functions -----------
# -------------------------------
def fetch_data(ticker, end_date, lookback_days):
    if end_date:
        end_dt = dt.datetime.combine(end_date + dt.timedelta(days=1), dt.time(0))
        start_dt = end_dt - dt.timedelta(days=lookback_days + 10)
        df = yf.download(ticker, start=start_dt.strftime("%Y-%m-%d"), end=end_dt.strftime("%Y-%m-%d"),
                         progress=False, auto_adjust=True, multi_level_index=False)
    else:
        period_str = f"{lookback_days}d"
        df = yf.download(ticker, period=period_str, interval="1d", progress=False, auto_adjust=True, multi_level_index=False)

    if df is None:
        return pd.DataFrame()
    df.index = pd.to_datetime(df.index).tz_localize(None)
    return df

def evaluate_signals_for_ticker(df, config):
    out = {"indicator_results": {}, "final_decision": False}
    if df.empty or len(df) < config["MIN_ROWS"]:
        out["reason"] = "insufficient_data"; return out

    close = df['Close']

    # EMA (custom if available)
    if HAVE_CUSTOM_IND and cust_calc_ema is not None:
        try:
            df['EMA_fast'] = cust_calc_ema(df, config["EMA_FAST_LENGTH"])
            df['EMA_slow'] = cust_calc_ema(df, config["EMA_SLOW_LENGTH"])
        except Exception:
            df['EMA_fast'] = ema(close, config["EMA_FAST_LENGTH"])
            df['EMA_slow'] = ema(close, config["EMA_SLOW_LENGTH"])
    else:
        df['EMA_fast'] = ema(close, config["EMA_FAST_LENGTH"])
        df['EMA_slow'] = ema(close, config["EMA_SLOW_LENGTH"])

    # SMAs (20/50 + 30)
    df['SMA_fast'] = sma(close, config["SMA_FAST_LENGTH"])
    df['SMA_slow'] = sma(close, config["SMA_SLOW_LENGTH"])
    df['SMA_30']   = sma(close, config.get("SMA30_LENGTH", 30))

    # Other indicators
    df['RSI'] = rsi(close, config["RSI_LENGTH"])
    df['VOL_SMA'] = df['Volume'].rolling(config["VOL_SMA_LENGTH"], min_periods=1).mean()
    df['OBV'] = obv(df)
    macd_df = macd(close, config["MACD_FAST"], config["MACD_SLOW"], config["MACD_SIGNAL"])
    df = df.join(macd_df)

    # ADX (fixed)
    try:
        df['ADX'] = adx(df, config["ADX_LENGTH"])
    except Exception:
        df['ADX'] = pd.Series(np.nan, index=df.index)

    # Supertrend (custom or fallback)
    try:
        if HAVE_CUSTOM_IND and cust_calc_supertrend is not None:
            df = cust_calc_supertrend(df, config["ST_LENGTH"], config["ST_MULTIPLIER"])
        else:
            st_df = supertrend_fallback(df, config["ST_LENGTH"], config["ST_MULTIPLIER"])
            df = df.join(st_df[['supertrend', 'supertrend_direction']], how='left')
    except Exception:
        st_df = supertrend_fallback(df, config["ST_LENGTH"], config["ST_MULTIPLIER"])
        df = df.join(st_df[['supertrend', 'supertrend_direction']], how='left')

    df = df.dropna().copy()
    if len(df) < 2:
        out["reason"] = "insufficient_after_indicator"; return out

    last = df.iloc[-1]
    prev = df.iloc[-2]

    results = {}
    inds = config["INDICATORS"]

    if inds.get("EMA_CROSS"):
        results['EMA_CROSS'] = (prev['EMA_fast'] <= prev['EMA_slow']) and (last['EMA_fast'] > last['EMA_slow'])
    if inds.get("SUPERTREND"):
        results['SUPERTREND'] = bool(last.get('supertrend_direction', -1) == 1)
    if inds.get("RSI"):
        results['RSI'] = bool(last['RSI'] <= config["RSI_BUY_THRESHOLD"])
    if inds.get("ADX"):
        results['ADX'] = bool(last.get('ADX', 0) >= config["ADX_THRESHOLD"])
    if inds.get("MACD"):
        results['MACD'] = bool((prev['macd'] <= prev['signal']) and (last['macd'] > last['signal']))
    if inds.get("BOLLINGER"):
        mid = df['Close'].rolling(config["BOLLINGER_LENGTH"], min_periods=1).mean()
        results['BOLLINGER'] = bool((df['Close'].iloc[-2] <= mid.iloc[-2]) and (df['Close'].iloc[-1] > mid.iloc[-1]))
    if inds.get("SMA"):
        results['SMA'] = bool((prev['SMA_fast'] <= prev['SMA_slow']) and (last['SMA_fast'] > last['SMA_slow']))
    if inds.get("SMA30"):  # Close crosses above SMA(30)
        results['SMA30'] = bool((prev['Close'] <= prev['SMA_30']) and (last['Close'] > last['SMA_30']))
    if inds.get("VOL_SMA"):
        results['VOL_SMA'] = bool(last['Volume'] > (last['VOL_SMA'] * config['VOL_MULTIPLIER']))
    if inds.get("OBV"):
        results['OBV'] = bool(df['OBV'].iloc[-1] > df['OBV'].iloc[-2])

    enabled_count = sum(1 for v in inds.values() if v)
    true_count = sum(1 for v in results.values() if v)
    mode = config.get("COMBINE_MODE", "all").lower()

    if enabled_count == 0:
        final = False
    elif mode == "all":
        final = (true_count == enabled_count)
    elif mode == "any":
        final = (true_count >= 1)
    elif mode == "majority":
        final = (true_count >= (enabled_count // 2) + 1)
    else:
        final = (true_count == enabled_count)

    out["indicator_results"] = results
    out["final_decision"] = bool(final)
    out["true_count"] = true_count
    out["enabled_count"] = enabled_count
    out["last_bar_date"] = df.index[-1]
    out["last_close"] = float(last['Close'])
    out["last_open"] = float(last['Open'])
    out["last_volume"] = float(last['Volume'])

    return out

def scan_universe(tickers, eval_date, config, output_root):
    eval_dt = dt.datetime.strptime(eval_date, "%Y-%m-%d").date() if eval_date else None
    run_date_str = eval_dt.strftime("%Y-%m-%d") if eval_dt else dt.date.today().strftime("%Y-%m-%d")
    out_dir = os.path.join(output_root, run_date_str); os.makedirs(out_dir, exist_ok=True)

    results = []
    details = []
    for t in tickers:
        if VERBOSE: print(f"Checking {t}...", end="", flush=True)
        try:
            df = fetch_data(t, eval_dt, config.get("YF_PERIOD_DAYS", 200))
            if df.empty or len(df) < config.get("MIN_ROWS", 30):
                if VERBOSE: print(" insufficient data."); details.append({"ticker": t, "status": "insufficient_data"}); continue

            if eval_dt:
                valid_idx = df.index[df.index.date <= eval_dt]
                if len(valid_idx) == 0:
                    if VERBOSE: print(" no trading data before eval date."); details.append({"ticker": t, "status": "no_trading_data_before_date"}); continue
                df = df.loc[:valid_idx[-1]]

            sig = evaluate_signals_for_ticker(df, config)
            rec = {
                "ticker": t,
                "decision": sig.get("final_decision", False),
                "enabled_count": sig.get("enabled_count", 0),
                "true_count": sig.get("true_count", 0),
                "last_bar_date": sig.get("last_bar_date", None),
                "last_close": sig.get("last_close", None),
                "reason": sig.get("reason", "")
            }
            results.append(rec)
            detail_record = {"ticker": t, **sig.get("indicator_results", {}), "final_decision": sig.get("final_decision", False)}
            details.append(detail_record)

            if VERBOSE:
                print(" -> ✅ Signal!" if rec["decision"] else " -> No signal.")
        except Exception as e:
            details.append({"ticker": t, "status": "error", "error": str(e)})
            if VERBOSE:
                print(f" -> Error: {e}")

    results_df = pd.DataFrame(results)
    details_df = pd.DataFrame(details)

    # Save outputs
    summary_path = os.path.join(out_dir, "summary_all_tickers.csv")
    details_path = os.path.join(out_dir, "details_indicators.csv")
    found_path = os.path.join(out_dir, "buy_signals.csv")

    results_df.to_csv(summary_path, index=False)
    details_df.to_csv(details_path, index=False)
    if not results_df.empty:
        results_df[results_df['decision'] == True][['ticker', 'last_bar_date', 'last_close']].to_csv(found_path, index=False)
    else:
        pd.DataFrame(columns=['ticker', 'last_bar_date', 'last_close']).to_csv(found_path, index=False)

    print(f"\nSaved outputs to {out_dir}")
    print(f"Total scanned: {len(tickers)}, Signals found: {len(results_df[results_df['decision'] == True])}")

    # ---- Telegram notifications ----
    tcfg = config.get("TELEGRAM", {})
    if tcfg.get("ENABLE", True):
        summary_msg = build_summary_message(results_df, details_df, config, run_date_str)
        send_telegram_message(summary_msg, tcfg)

        if tcfg.get("SEND_EACH_TRADE", False):
            for msg in build_per_trade_messages(results_df, details_df, config, run_date_str):
                send_telegram_message(msg, tcfg)

    return {"output_dir": out_dir, "summary_path": summary_path, "details_path": details_path, "found_path": found_path}


# -------------------------------
# --------- MAIN ---------------
# -------------------------------
if __name__ == "__main__":
    scan_universe(STOCKS_TO_SCAN, EVAL_DATE, CONFIG, OUTPUT_ROOT)


Checking SBICARD.NS... -> No signal.
Checking BDL.NS... -> No signal.
Checking INDHOTEL.NS... -> No signal.
Checking BSE.NS... -> No signal.
Checking NYKAA.NS... -> No signal.
Checking BAJFINANCE.NS... -> No signal.
Checking PAYTM.NS... -> No signal.
Checking SOLARINDS.NS... -> No signal.
Checking CHOLAFIN.NS... -> No signal.
Checking UNITDSPR.NS... -> No signal.
Checking DIVISLAB.NS... -> No signal.
Checking MUTHOOTFIN.NS... -> No signal.
Checking BHARTIARTL.NS... -> No signal.
Checking ICICIBANK.NS... -> No signal.
Checking MAZDOCK.NS... -> No signal.
Checking SHREECEM.NS... -> No signal.
Checking DIXON.NS... -> No signal.
Checking PERSISTENT.NS... -> No signal.
Checking SRF.NS... -> No signal.
Checking TVSMOTOR.NS... -> No signal.
Checking SBILIFE.NS... -> ✅ Signal!
Checking MAXHEALTH.NS... -> No signal.
Checking MFSL.NS... -> No signal.
Checking COFORGE.NS... -> No signal.
Checking HDFCLIFE.NS... -> No signal.
Checking INDIGO.NS... -> No signal.
Checking KOTAKBANK.NS... -> No signa